In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader("climate_def_cause.txt", encoding="utf-8")
document = loader.load()

In [2]:
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=50)
docs = splitter.split_documents(document)

# print(chunks[0])

In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

embed = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

texts = [doc.page_content for doc in docs]
embeddings = embed.encode(texts)

index = faiss.IndexFlatL2(embeddings[0].shape[0])
index.add(np.array(embeddings))

c:\Users\darsh\Documents\ml\rag\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def retriever(query, k = 3):
    query_embeddings = embed.encode([query])
    dist, ind = index.search(np.array(query_embeddings), k)
    return [texts[i] for i in ind[0]]

In [5]:
def get_context(chunks, tokenizer, max_tokens=450):
    context = ""
    token_count = 0

    for chunk in chunks:
        tokens = tokenizer(chunk, return_tensors="pt")["input_ids"][0]
        if token_count + len(tokens) > max_tokens:
            break
        context += chunk + "\n"
        token_count += len(tokens)

    return context


In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model1 = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
tokeniser = AutoTokenizer.from_pretrained("google/flan-t5-large")

def gen_answer(query):
    context_chunks = retriever(query)
    context = get_context(context_chunks, tokeniser)

    prompts = f"""You are a science expert. Based on the following context, answer the user's question clearly and correctly.

    Context:
    {context}

    Question: {query}
    Answer:"""
    
    inputs = tokeniser(prompts, return_tensors="pt", max_length=512, truncation=True)
    outputs = model1.generate(**inputs, max_new_tokens=200)
    return tokeniser.decode(outputs[0], skip_special_tokens=True)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Error while downloading from https://huggingface.co/google/flan-t5-large/resolve/main/model.safetensors: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...


OSError: google/flan-t5-large does not appear to have a file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt or flax_model.msgpack.

In [69]:
print("Query: ", "explain greenhouse effect")
response = gen_answer("explain greenhouse effect")
print("Answer: ", response)

Query:  explain greenhouse effect
Answer:  Water vapour and clouds
